Task 1: Third-order letter approximation model

In [39]:
import collections

In [40]:
with open('/workspaces/emergingtechnologies/Data/Frankenstien.txt', 'r') as f:
    text = f.read()

In [41]:
chars = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ ')
text = text.upper()
cleaned = ''.join(c for c in text if c in chars)


In [44]:
count = collections.Counter(cleaned)
count.most_common()

[(' ', 71747),
 ('E', 46094),
 ('T', 30379),
 ('A', 26743),
 ('O', 25254),
 ('I', 24577),
 ('N', 24359),
 ('S', 21173),
 ('R', 20876),
 ('H', 19763),
 ('D', 16858),
 ('L', 12722),
 ('M', 10545),
 ('U', 10412),
 ('C', 9275),
 ('F', 8722),
 ('Y', 7923),
 ('W', 7653),
 ('P', 6134),
 ('G', 5980),
 ('B', 5021),
 ('V', 3829),
 ('K', 1760),
 ('X', 677),
 ('J', 502),
 ('Q', 324),
 ('Z', 213)]

In [45]:
count.keys()

dict_keys(['T', 'H', 'E', ' ', 'P', 'R', 'O', 'J', 'C', 'G', 'U', 'N', 'B', 'K', 'F', 'A', 'S', 'I', 'M', 'D', 'Y', 'W', 'L', 'V', 'Z', 'X', 'Q'])

In [46]:
count.values()

dict_values([30379, 19763, 46094, 71747, 6134, 20876, 25254, 502, 9275, 5980, 10412, 24359, 5021, 1760, 8722, 26743, 21173, 24577, 10545, 16858, 7923, 7653, 12722, 3829, 213, 677, 324])

In [49]:
chars, weights = count.keys(), count.values()
weights

dict_values([30379, 19763, 46094, 71747, 6134, 20876, 25254, 502, 9275, 5980, 10412, 24359, 5021, 1760, 8722, 26743, 21173, 24577, 10545, 16858, 7923, 7653, 12722, 3829, 213, 677, 324])